![Elastacloud](https://images.squarespace-cdn.com/content/v1/5e3a928770897e641dceeaab/1589874757659-EY91JGOYIF31K7X9549S/black+and+blue+logo.png?format=200w)

# Microsoft SQL ODBC Driver

If you need to use the Microsoft SQL ODBC driver from your jobs, such as via PyODBC for retrieving scalar values, then you can do so by using the Microsoft ODBC driver for linux. This is documentation in a few places online and usually results in an init script which looks like the following.

```bash
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/18.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17
```

## The issue

At the time of writing Microsoft experienced a 20+ hour outage of their packages repo, meaning that any cluster needing to install the driver would fail, the packages were not available until the issue had been resolved. There are also times when the repo is being synchronized and so the packages are not available. If a job is initiated at this time then the init scripts will again fail.

## The solution

This notebook looks to implement a method which offers some resiliance to this issue. It utilises 3 folders:

Folder   | Description
-------- | -----------
next     | Latest versions of packages are downloaded here
current  | The most recent successful downloads which the init script uses
previous | The last successful download of packages. These are maintained in case a rollback to the previous versions is required

The process is as follows

1. Download the ODBC driver and dependent packages to a `next` directory
1. If all of the packages have downloaded successfully then:
  1. Remove the `previous` directory
  1. Move the `current` directory to `previous`
  1. Move the `next` directory to `current`
  1. Update the init script (in case you change your mind on directory names)
1. If the packages did not download successfully, then leave `previous` and `current` as they currently are so that the init scripts continue to function

### Setting up

This notebook is executed independently of the init scripts and should be run as a job. After a first successful execution the job could subsequently fail because of an outage without then affecting the init script

In [0]:
import shutil
from pathlib import Path
from typing import List
from urllib.parse import urlparse

import requests

## File listings

These files are the ones required to be downloaded from https://packages.microsoft.com

In [0]:
sources: List[str] = [
    "https://packages.microsoft.com/ubuntu/18.04/prod/pool/main/m/msodbcsql17/msodbcsql17_17.7.2.1-1_amd64.deb"
    , "https://packages.microsoft.com/ubuntu/18.04/prod/pool/main/u/unixodbc/odbcinst_2.3.7_amd64.deb"
    , "https://packages.microsoft.com/ubuntu/18.04/prod/pool/main/u/unixodbc/unixodbc-dev_2.3.7_amd64.deb"
    , "https://packages.microsoft.com/ubuntu/18.04/prod/pool/main/u/unixodbc/odbcinst1debian2_2.3.7_amd64.deb"
    , "https://packages.microsoft.com/ubuntu/18.04/prod/pool/main/u/unixodbc/libodbc1_2.3.7_amd64.deb"
    , "https://packages.microsoft.com/ubuntu/18.04/prod/pool/main/u/unixodbc/unixodbc_2.3.7_amd64.deb"
]

## Directory configuration

Here we define the 3 directories and set them up ready for downloading

In [0]:
root: str = "/dbfs/FileStore/dependencies"
previous: Path = Path(f"{root}/previous")
current: Path = Path(f"{root}/current")
new: Path = Path(f"{root}/next")

In [0]:
if new.exists():
    shutil.rmtree(new)

current.mkdir(parents=True, exist_ok=True)
new.mkdir(parents=True, exist_ok=True)

downloaded: List[str] = []

## Download

Download the packages, if successful then replace the `current` and `previous` directories with modified packages and re-weite the init script.

**N.B.** If you don't want to keep overwriting the init script then simply remove the `dbutils.fs.put` command.

In [0]:
try:
    session: requests.Session
    with requests.Session() as session:
        for url in sources:
            output_path: Path = new / urlparse(url).path.split("/")[-1]
            resp: requests.Response = session.get(url)
            if resp.ok:
                output_path.write_bytes(resp.content)
                downloaded.append(str(current / output_path.name))
            else:
                raise Exception(f"Unable to download from {url}, received {resp.status_code}")

    if previous.exists():
        shutil.rmtree(previous)

    shutil.move(current, previous)
    shutil.move(new, current)
    
    dbutils.fs.put("dbfs:/FileStore/init_scripts/mssql-driver.sh", f"""echo "Installing SQL ODBC Driver and dependencies"
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install {' '.join(downloaded)}

echo "Installing Python dev libraries"
sudo apt-get -q -y install python3-dev

echo "Installing PyODBC"
/databricks/python/bin/pip install pyodbc""", True)
except Exception as e:
    shutil.rmtree(new)
    raise(e)